In [1]:
%%configure -f
{
    "conf": {
        "spark.app.name": "datavirus_final"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7611,application_1589299642358_2106,pyspark,idle,Link,Link,
7632,application_1589299642358_2126,pyspark,idle,Link,Link,
7633,application_1589299642358_2127,pyspark,idle,Link,Link,
7644,application_1589299642358_2140,pyspark,idle,Link,Link,
7670,application_1589299642358_2166,pyspark,busy,Link,Link,
7674,application_1589299642358_2170,pyspark,idle,Link,Link,
7676,application_1589299642358_2172,pyspark,busy,Link,Link,
7677,application_1589299642358_2173,pyspark,busy,Link,Link,
7681,application_1589299642358_2177,pyspark,idle,Link,Link,
7683,application_1589299642358_2179,pyspark,idle,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7701,application_1589299642358_2197,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
import pyspark.sql.functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
timetable = spark.read.orc("/user/gflueck/timetable_zurich.orc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
timetable.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|
+--------------------+------------+--------------+-------+-------------+
|3.TA.26-453-j19-1...|    16:34:00|      16:34:00|8576279|            5|
|3.TA.26-453-j19-1...|    16:36:00|      16:36:00|8576280|            6|
|3.TA.26-453-j19-1...|    16:38:00|      16:38:00|8590878|            7|
|3.TA.26-454-j19-1...|    18:21:00|      18:21:00|8595714|            1|
|3.TA.26-454-j19-1...|    18:23:00|      18:23:00|8590758|            2|
+--------------------+------------+--------------+-------+-------------+
only showing top 5 rows

In [7]:
from pyspark.sql import Window
trip_window = Window.partitionBy('trip_id').orderBy(F.asc('arrival_time'))
trip_rank = F.rank().over(trip_window).alias('new_stop')
begin =timetable.select('*', trip_rank).alias('begin').orderBy('trip_id','arrival_time').fillna(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
begin.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+--------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|new_stop|
+--------------------+------------+--------------+-------+-------------+--------+
|1.TA.1-1-E-j19-1.1.H|    05:38:00|      05:38:00|8590317|           22|       1|
|1.TA.1-1-E-j19-1.1.H|    05:39:00|      05:39:00|8590314|           23|       2|
|1.TA.1-1-E-j19-1.1.H|    05:40:00|      05:40:00|8578679|           24|       3|
|  1.TA.1-1-j19-1.1.H|    05:19:00|      05:19:00|8503000|            1|       1|
| 1.TA.1-10-j19-1.1.R|    06:03:00|      06:03:00|8590274|            1|       1|
| 1.TA.1-10-j19-1.1.R|    06:04:00|      06:04:00|8590272|            2|       2|
| 1.TA.1-10-j19-1.1.R|    06:05:00|      06:05:00|8590276|            3|       3|
| 1.TA.1-10-j19-1.1.R|    06:06:00|      06:06:00|8590269|            4|       4|
|1.TA.1-11-B-j19-1...|    06:16:00|      06:16:00|8594304|            3|       1|
|1.TA.1-11-B-j19

In [10]:
ids_counts = begin.groupBy('trip_id').count()
ids_trips = ids_counts.where(ids_counts['count']>1).select('trip_id').distinct()
df = begin.join(ids_trips, "trip_id").orderBy('trip_id','arrival_time','new_stop')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+--------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|new_stop|
+--------------------+------------+--------------+-------+-------------+--------+
|1.TA.1-1-E-j19-1.1.H|    05:38:00|      05:38:00|8590317|           22|       1|
|1.TA.1-1-E-j19-1.1.H|    05:39:00|      05:39:00|8590314|           23|       2|
|1.TA.1-1-E-j19-1.1.H|    05:40:00|      05:40:00|8578679|           24|       3|
| 1.TA.1-10-j19-1.1.R|    06:03:00|      06:03:00|8590274|            1|       1|
| 1.TA.1-10-j19-1.1.R|    06:04:00|      06:04:00|8590272|            2|       2|
| 1.TA.1-10-j19-1.1.R|    06:05:00|      06:05:00|8590276|            3|       3|
| 1.TA.1-10-j19-1.1.R|    06:06:00|      06:06:00|8590269|            4|       4|
|1.TA.1-11-B-j19-1...|    06:16:00|      06:16:00|8594304|            3|       1|
|1.TA.1-11-B-j19-1...|    06:17:00|      06:17:00|8594310|            4|       2|
|1.TA.1-11-B-j19

In [19]:
to_station = df.drop('departure_time','stop_sequence').withColumn('new_stop', df.new_stop -1).alias('to_station')
from_station = df.drop('arrival_time','stop_sequence').join(to_station, on=['new_stop','trip_id']).orderBy('trip_id','new_stop')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
schedule = from_station.toDF("Stop_sequence","Trip_ID","Start_Time","Start_Station","Stop_Time","Stop_Station").

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
schedule.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+----------+-------------+---------+------------+
|Stop_sequence|             Trip_ID|Start_Time|Start_Station|Stop_Time|Stop_Station|
+-------------+--------------------+----------+-------------+---------+------------+
|            1|1.TA.1-1-E-j19-1.1.H|  05:38:00|      8590317| 05:39:00|     8590314|
|            2|1.TA.1-1-E-j19-1.1.H|  05:39:00|      8590314| 05:40:00|     8578679|
|            1| 1.TA.1-10-j19-1.1.R|  06:03:00|      8590274| 06:04:00|     8590272|
|            2| 1.TA.1-10-j19-1.1.R|  06:04:00|      8590272| 06:05:00|     8590276|
|            3| 1.TA.1-10-j19-1.1.R|  06:05:00|      8590276| 06:06:00|     8590269|
|            1|1.TA.1-11-B-j19-1...|  06:16:00|      8594304| 06:17:00|     8594310|
|            2|1.TA.1-11-B-j19-1...|  06:17:00|      8594310| 06:18:00|     8594307|
|            3|1.TA.1-11-B-j19-1...|  06:18:00|      8594307| 06:19:00|     8590317|
|            4|1.TA.1-11-B-j19-1...|  06:19:00|      8590317| 06:

In [23]:
schedule.write.format("orc").save("/user/datavirus/schedule_zurich.orc", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…